<a href="https://colab.research.google.com/github/ArkaEnergyAI/ArkaAIProject/blob/main/inference/roof_outline_and_obstructions_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Roof outline and Obstructions Inference Notebook

## This notebook shows how to run inference to get center roof outline & obstructions bounding boxes

## **Authors**
**Ashwin Sarathy**
: Worked extensively on dataset building & Simplification Algorithm

**Shaurya Tripathi**
: Worked on Google Image Generation, json update

**Shiva Srivastava**
: Concieved this inference flow, trained segmentation and detection models

## Roboflow used for dataset generation & versioning, model versioning.

### Roboflow install

In [ ]:
# Execute this cell once to install roboflow
!pip install roboflow
from IPython import display
display.clear_output()

##### Other miscellaneous imports

In [ ]:
# Bunch of imports
from roboflow import Roboflow
import os
from IPython.display import display, Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
import json
from IPython.lib.display import exists
from PIL import Image
import time

##### General house keeping - declaring all paths here

In [ ]:
HOME = os.getcwd()
print(HOME)
RESULTS_DIR = "../results/"
print(RESULTS_DIR)
os.makedirs(RESULTS_DIR, exist_ok=True)
IMAGES_DIR = "../Images/"
print(IMAGES_DIR)
assert(os.path.isdir(IMAGES_DIR))

## Functions to access Segmentation & Detection models, and Douglas Peucker Algorithm for polyline simplification

### YOLOv8 instance segmentation model

#### Load model

In [ ]:
# Load Segmentation model
# Update version number if new version of model generated
def load_segmentation_model(api_key="9pyHdiM1tENb1DwODp1M", project_name="arkasegmentationproject", version=7):
    from roboflow import Roboflow
    rf = Roboflow(api_key=api_key)
    project = rf.workspace().project(project_name)
    seg_model = project.version(version).model
    return seg_model

#### Display Segmented Image & segmentation json save

In [ ]:
def outline_pred_and_json(in_path, out_dir, seg_model):
    os.makedirs(out_dir, exist_ok=True)

    seg_pred_json = seg_model.predict(in_path, confidence=50).json()
    # f_name = os.path.splitext(os.path.basename(in_path))[0]
    # with open(out_dir + f_name + "_seg_pred_orig" + ".json", "w") as json_file:
    #     json.dump(seg_pred_json, json_file)

    return seg_pred_json

#### Get the points dict from json

In [ ]:

def return_points(out_json): return out_json["predictions"][0]["points"]

### Douglas Peucker Polyline Simplification algorithm

##### Recommended epsilon values between 10 to 15

In [ ]:
def distance(p1, p2):
    return math.sqrt((p1['x'] - p2['x']) ** 2 + (p1['y'] - p2['y']) ** 2)


def perpendicular_distance(point, line_start, line_end):
    numerator = abs((line_end['y'] - line_start['y']) * point['x'] - (line_end['x'] - line_start['x'])
                    * point['y'] + line_end['x'] * line_start['y'] - line_end['y'] * line_start['x'])
    denominator = distance(line_start, line_end)
    return numerator / denominator

# Douglas Peucker Algorithm


def douglas_peucker(points, epsilon):
    if len(points) <= 2:
        return points

    dmax = 0
    index = 0

    for i in range(1, len(points) - 1):
        d = perpendicular_distance(points[i], points[0], points[-1])
        if d > dmax:
            index = i
            dmax = d

    if dmax > epsilon:
        result = (
            douglas_peucker(points[:index + 1], epsilon)[:-1] +
            douglas_peucker(points[index:], epsilon)
        )
    else:
        result = [points[0], points[-1]]

    return result

##### Update Roof outline json with simplified points

In [ ]:
def update_seg_pred_simplified_points(seg_pred_json, simplified_points):
    seg_pred_json["predictions"][0]["points"] = simplified_points
    return seg_pred_json


def updated_seg_json(seg_pred_json, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    img_path = seg_pred_json["predictions"][0]['image_path']
    f_name = os.path.splitext(os.path.basename(img_path))[0]
    with open(out_dir + f_name + "_seg_pred_simplified" + ".json", "w") as json_file:
        json.dump(seg_pred_json, json_file)


def print_simplified_points_img(img_path, simplified_points):
    image = cv2.imread(img_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    x_coords = [point['x'] for point in simplified_points]
    y_coords = [point['y'] for point in simplified_points]

    plt.figure(figsize=(8, 8))
    plt.imshow(image_rgb)

    plt.plot(x_coords, y_coords, '-ro', label='Simplified Path', markersize=3)

    plt.plot([x_coords[-1], x_coords[0]],
             [y_coords[-1], y_coords[0]], '-ro', markersize=3)

    plt.legend()
    plt.show()


def print_simplified_points_img_obs(img_path, simplified_points):
    image = cv2.imread(img_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    x_coords = [point['x'] for point in simplified_points]
    y_coords = [point['y'] for point in simplified_points]

    plt.figure(figsize=(8, 8))
    plt.imshow(image_rgb)

    plt.plot(x_coords, y_coords, '-bo',
             label="roof-outline + obstructions", markersize=3)

    plt.plot([x_coords[-1], x_coords[0]],
             [y_coords[-1], y_coords[0]], '-bo', markersize=3)

    plt.legend()
    plt.show()

Margin: this is the pixel buffers around the bounding box

In [ ]:
def cut_bounding_box(img_path, seg_pred_json, out_dir, margin=10):
    image_rgb = cv2.imread(img_path)
    for bounding_box in seg_pred_json["predictions"]:
        x0 = bounding_box['x'] - bounding_box['width'] / 2 - margin
        x1 = bounding_box['x'] + bounding_box['width'] / 2 + margin
        y0 = bounding_box['y'] - bounding_box['height'] / 2 - margin
        y1 = bounding_box['y'] + bounding_box['height'] / 2 + margin

        center_roof_bb = image_rgb[int(y0):int(y1), int(x0):int(x1)]

    os.makedirs(out_dir, exist_ok=True)
    f_name = os.path.basename(img_path)
    out_path = out_dir+"center_roof_"+f_name
    cv2.imwrite(out_path, center_roof_bb)
    

### YOLOv8 Object Detection model for Obstruction detection

##### Load Obstruction detection model

In [ ]:
def load_detection_model(api_key="9pyHdiM1tENb1DwODp1M", project_name="roofobstructionsdetect", version=11):
    from roboflow import Roboflow
    rf = Roboflow(api_key=api_key)
    project = rf.workspace().project(project_name)
    det_model = project.version(version).model
    return det_model

##### Get obstruction prediction and json

In [ ]:

def get_obstruction_prediction_json(obs_model, img_path, out_dir):
    obstr_preds = obs_model.predict(img_path, confidence=40, overlap=30).json()

    os.makedirs(out_dir, exist_ok=True)
    # out_path = out_dir+"obs_"+os.path.basename(img_path)
    # obs_model.predict(img_path, confidence=50).save(out_path)
    # f_name = os.path.splitext(os.path.basename(img_path))[0]
    # with open(out_dir + f_name + "_obs_pred" + ".json", "w") as json_file:
    #     json.dump(obstr_preds, json_file)

    return obstr_preds

###  JSON and IMAGE functions

##### Overlay roof outline and obstructions on the Image

In [ ]:
def overlay_on_original_img(seg_pred_json, obs_pred_json, img_path, out_dir, margin=10):
    orig_x = seg_pred_json['predictions'][0]['x'] - \
        seg_pred_json['predictions'][0]['width']/2 - margin
    orig_y = seg_pred_json['predictions'][0]['y'] - \
        seg_pred_json['predictions'][0]['height']/2 - margin

    image_rgb = cv2.imread(img_path)
    for bounding_box in obs_pred_json["predictions"]:
        x0 = orig_x + bounding_box['x'] - bounding_box['width'] / 2
        x1 = orig_x + bounding_box['x'] + bounding_box['width'] / 2
        y0 = orig_y + bounding_box['y'] - bounding_box['height'] / 2
        y1 = orig_y + bounding_box['y'] + bounding_box['height'] / 2

        start_point = (int(x0), int(y0))
        end_point = (int(x1), int(y1))
        cv2.rectangle(image_rgb, start_point, end_point,
                      color=(0, 0, 255), thickness=2)

    os.makedirs(out_dir, exist_ok=True)
    f_name = os.path.basename(img_path)
    out_path = out_dir+"final_output_"+f_name
    cv2.imwrite(out_path, image_rgb)

#### Creating two JSON files

##### Predicted segment points + obstruction bounding boxes mapped to original image

In [ ]:
def output_json_all_points_obs_orig(img_path, seg_pred_json, obs_pred_json, margin=10, out_dir=RESULTS_DIR):
    os.makedirs(out_dir, exist_ok=True)

    orig_x = seg_pred_json['predictions'][0]['x'] - \
        seg_pred_json['predictions'][0]['width']/2 - margin
    orig_y = seg_pred_json['predictions'][0]['y'] - \
        seg_pred_json['predictions'][0]['height']/2 - margin

    for i in range(len(obs_pred_json["predictions"])):
        obs_pred_json['predictions'][i]['x'] = orig_x + \
            obs_pred_json['predictions'][i]['x']
        obs_pred_json['predictions'][i]['y'] = orig_y + \
            obs_pred_json['predictions'][i]['y']
        obs_pred_json['predictions'][i]['image_path'] = seg_pred_json['predictions'][0]['image_path']

    combined_json = {
        'predictions': seg_pred_json['predictions'] + obs_pred_json['predictions'],
        'image': seg_pred_json['image']
    }

    f_name = os.path.splitext(os.path.basename(img_path))[0]
    with open(out_dir + f_name + "_seg_obs_pred_orig" + ".json", "w") as json_file:
        json.dump(combined_json, json_file)

##### Simplified segment points + obstruction bounding boxes mapped to original image

In [ ]:
def output_json_all_points_obs_simplified(img_path, seg_pred_json, obs_pred_json, margin=10, out_dir=RESULTS_DIR):
    os.makedirs(out_dir, exist_ok=True)

    orig_x = seg_pred_json['predictions'][0]['x'] - \
        seg_pred_json['predictions'][0]['width']/2 - margin
    orig_y = seg_pred_json['predictions'][0]['y'] - \
        seg_pred_json['predictions'][0]['height']/2 - margin

    for i in range(len(obs_pred_json["predictions"])):
        obs_pred_json['predictions'][i]['x'] = orig_x + \
            obs_pred_json['predictions'][i]['x']
        obs_pred_json['predictions'][i]['y'] = orig_y + \
            obs_pred_json['predictions'][i]['y']
        obs_pred_json['predictions'][i]['image_path'] = seg_pred_json['predictions'][0]['image_path']

    combined_json = {
        'predictions': seg_pred_json['predictions'] + obs_pred_json['predictions'],
        'image': seg_pred_json['image']
    }

    f_name = os.path.splitext(os.path.basename(img_path))[0]
    with open(out_dir + f_name + "_seg_obs_pred_simplified" + ".json", "w") as json_file:
        json.dump(combined_json, json_file)

##### Check for Obstructions display

In [ ]:
def verify_obs_from_saved_json(img_path, seg_obs_pred_json):
    image_bgr = cv2.imread(img_path)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    for i in range(1, len(seg_obs_pred_json["predictions"])):
        x0 = seg_obs_pred_json["predictions"][i]['x'] - \
            seg_obs_pred_json["predictions"][i]['width'] / 2
        x1 = seg_obs_pred_json["predictions"][i]['x'] + \
            seg_obs_pred_json["predictions"][i]['width'] / 2
        y0 = seg_obs_pred_json["predictions"][i]['y'] - \
            seg_obs_pred_json["predictions"][i]['height'] / 2
        y1 = seg_obs_pred_json["predictions"][i]['y'] + \
            seg_obs_pred_json["predictions"][i]['height'] / 2

        start_point = (int(x0), int(y0))
        end_point = (int(x1), int(y1))

        image_rgb = cv2.rectangle(
            image_rgb, start_point, end_point, color=(0, 255, 0), thickness=2)

    plt.figure(figsize=(8, 8))
    plt.imshow(image_rgb)

## All setup! Use the code below to invoke functions and get predictions


#### Load segmentation and detection models

In [ ]:
try:
    seg_model = load_segmentation_model()
    obs_model = load_detection_model()
except Exception as e:
    raise (f"Models didn't load properly! {e}")

#### Time the roof-outline and obstruction determination

In [ ]:
# Set the image name
#img = IMAGES_DIR + "12321KosichplSaratoga.jpg"
#img = IMAGES_DIR + "12332ObradDrSaratoga.jpg"
img = IMAGES_DIR + "701SaranacDrSunnyvale.jpg"
img_name = os.path.basename(img)

stFull = time.time()
# Segmentation prediction
st = time.time()
seg_pred_json = outline_pred_and_json(img, RESULTS_DIR, seg_model)
et = time.time()
print('Segmentation Execution time:', (et-st), 'seconds')

# Segmentation results, proceed further only if center roof identified
if seg_pred_json:
    points = return_points(seg_pred_json)
    # simplify the roof-outline predictions
    epsilon = 5
    st = time.time()
    simplified_points = douglas_peucker(points, epsilon)
    et = time.time()
    print('Simplification Algo Execution time:', (et-st), 'seconds')

    seg_simp_json = update_seg_pred_simplified_points(
    seg_pred_json, simplified_points)

    #updated_seg_json(seg_simp_json, RESULTS_DIR)
    #print_simplified_points_img(img, simplified_points)

    # Crop the center roof
    cut_bounding_box(img, seg_pred_json, RESULTS_DIR)

    # Obstruction model
    st = time.time()
    obs_pred_json = get_obstruction_prediction_json(
        obs_model, RESULTS_DIR+"center_roof_"+img_name, RESULTS_DIR)
    et = time.time()
    print('Detection Execution time:', (et-st), 'seconds')
    if not obs_pred_json:
        print("Obstruction JSON is empty! Try again")

    overlay_on_original_img(seg_pred_json, obs_pred_json, img, RESULTS_DIR)
    etFull = time.time()
    print('Execution time:', (etFull-stFull), 'seconds')

    print_simplified_points_img_obs(
        RESULTS_DIR+"final_output_"+img_name, simplified_points)


    # # Save final JSONs
    output_json_all_points_obs_orig(
        img, seg_pred_json, obs_pred_json)
    
    output_json_all_points_obs_simplified(
        img, seg_simp_json, obs_pred_json)
    
    # verify the obstructions in json file
    # f_name = os.path.splitext(os.path.basename(img))[0]
    # print(f_name)
    # with open(RESULTS_DIR+f_name+"_seg_obs_pred_orig.json", "r") as file2:
    #     seg_obs_pred_orig_finaljson = json.load(file2)
    
    # verify_obs_from_saved_json(img, seg_obs_pred_orig_finaljson)
    
else:
    print("Segmentation Prediction JSON is empty! Try again")


### Use the code block down below to obtain JSON

In [ ]:
# Set the image name
#img = IMAGES_DIR + "1610FilareeCtCarlsbad.jpg"
#img = IMAGES_DIR + "18681KosichDrSaratoga.jpg"
#img = IMAGES_DIR + "Fremont_530.jpg"
img = IMAGES_DIR + "12332ObradDrSaratoga.jpg"
#img = IMAGES_DIR + "701SaranacDrSunnyvale.jpg"
img_name = os.path.basename(img)


seg_pred_json = outline_pred_and_json(img, RESULTS_DIR, seg_model)

# Segmentation results, proceed further only if center roof identified
if seg_pred_json: 
    points = return_points(seg_pred_json) 
    # Crop the center roof
    cut_bounding_box(img, seg_pred_json, RESULTS_DIR)

    # Obstruction model
    
    obs_pred_json = get_obstruction_prediction_json(
        obs_model, RESULTS_DIR+"center_roof_"+img_name, RESULTS_DIR)
    
    if not obs_pred_json:
        print("Obstruction JSON is empty! Try again")

    overlay_on_original_img(seg_pred_json, obs_pred_json, img, RESULTS_DIR)
    

    print_simplified_points_img_obs(
        RESULTS_DIR+"final_output_"+img_name, points)


    # # Save final JSONs
    output_json_all_points_obs_orig(
        img, seg_pred_json, obs_pred_json)
    
    
    
else:
    print("Segmentation Prediction JSON is empty! Try again")